In [12]:
import sys
import os

# Chemin absolu du répertoire racine du projet
project_root = '/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/'

# Ajouter le répertoire src au chemin de recherche des modules
src = os.path.join(project_root, '.')
sys.path.append(src)



In [13]:
import pandas as pd
import numpy as np
from src.data_concatenation.concatenate import build_macro_table
from src.data_preprocessing.preprocessing import preprocess_dataframe
from src.data_preprocessing.topificator import TopicExtractor
from src.database_management.database_initializer import DatabaseInitializer, wait_for_postgresql
from src.database_management.dataframe_to_transactionalDB import insert_data_from_dataframe
from src.database_management.migrate_to_decisionalDB import migration_to_decisionalDB
from src.database_management.db_models import Base
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from datetime import datetime, timedelta
import psycopg2, os

from tqdm import tqdm

CURRENT_DATE = datetime.now().strftime("%Y-%m-%d")
DATA_PATH = os.path.join(project_root, 'data')
CITIES_PATH = os.path.join(project_root, 'data', 'utils', 'countries_cities-full.json')
RAW_SAVE_PATH = os.path.join(project_root, 'data', 'temp')
PARCKET_PATH = os.path.join(project_root, 'data', 'parquet', CURRENT_DATE)


concatenate_parquets = build_macro_table(DATA_PATH)

# --------------------------------------------------------
# 0.3 Preprocessing of concatenate parquets
# --------------------------------------------------------
# df = preprocess_dataframe(concatenate_parquets)


/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/data/parquet/2024-07-30/Morocco/Nador.parquet: 90 rows
Macro table saved at: /home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/data/temp/macro_table/macro_table.parquet
Macro table saved as CSV at : /home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/data/temp/macro_table/macro_table.csv


In [14]:
len(concatenate_parquets.columns)

17

In [15]:

# --------------------------------------------------------
# 0.3 Preprocessing of concatenate parquets
# --------------------------------------------------------
df = preprocess_dataframe(concatenate_parquets)

/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/./src/data_preprocessing/preprocessing.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna({


Topic : {'topics': [('application mobile', 'Negative', ['bug', 'difficulte de connexion'])]}
DIRECTORY : /home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/data/processed/2024-08-06


  1%|▏         | 1/73 [00:53<1:04:30, 53.75s/it]

Topic 0 : Pays Ahsan ::: {'topics': [('Gestion des comptes', 'Positive', ['facile à utiliser', 'intuitive']), ('Sécurité', 'Neutral', ['pas de problème signalé', "mais pas d'information sur la sécurité"])]}


  3%|▎         | 2/73 [01:33<54:00, 45.65s/it]  

...

Topic 87 : Prestation : 0
Solutions : 0
Attention : 0 ::: {'topics': [('Sécurité', 'Negative', ['fraude', 'problèmes']), ('Service client', 'Neutral', ['inexistants', 'désagréables'])]}


 99%|█████████▊| 72/73 [48:26<00:30, 30.49s/it]

Topic 88 : Très bien situé, personnel excellent. Il y a parfois des problèmes sur Internet ::: {'topics': [('Localisation et accessibilité', 'Positive', ['bien situé'])]}


100%|██████████| 73/73 [48:53<00:00, 40.19s/it]

Topic 89 : service 0 ::: {'topics': [("Temps d'attente", 'Nan', [])]}


In [16]:
df.columns

Index(['Country', 'Town', 'Bank_Name', 'Bank_Phone_number', 'Bank_Address',
       'Bank_Website', 'Reviewer_Name', 'Reviewer_Star', 'Reviewer_Text',
       'Reviewer_Publish_Date', 'Reviewer_Like_Reaction',
       'Reviewer_Profile_Link', 'Reviewer_Owner_Reply',
       'Reviewer_Owner_Reply_Date', 'Topic', 'Sentiment', 'Sub_Topic'],
      dtype='object')

In [17]:
# all df
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# df


In [18]:
df.isna().sum()

Country                       0
Town                          0
Bank_Name                     0
Bank_Phone_number             0
Bank_Address                  0
Bank_Website                  0
Reviewer_Name                 0
Reviewer_Star                 0
Reviewer_Text                 0
Reviewer_Publish_Date         0
Reviewer_Like_Reaction        0
Reviewer_Profile_Link         0
Reviewer_Owner_Reply          2
Reviewer_Owner_Reply_Date     0
Topic                        73
Sentiment                    73
Sub_Topic                    73
dtype: int64

In [19]:
df.head()

,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Reviewer_Name,Reviewer_Star,Reviewer_Text,Reviewer_Publish_Date,Reviewer_Like_Reaction,Reviewer_Profile_Link,Reviewer_Owner_Reply,Reviewer_Owner_Reply_Date,Topic,Sentiment,Sub_Topic
0,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Habib Chouaa,5,Pays Ahsan,2016-08-08 18:50:25.525147,0,https://www.google.com/maps/contrib/1006948281...,Nothing,1970-01-01,NaN,NaN,NaN
1,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Mohamed Nador,4,Berger,2016-08-08 18:50:25.525158,0,https://www.google.com/maps/contrib/1165746157...,Nothing,1970-01-01,NaN,NaN,NaN
2,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Hafaimya,3,Expérience neutre,2021-08-07 18:50:25.525160,0,https://www.google.com/maps/contrib/1027703754...,Nothing,1970-01-01,NaN,NaN,NaN
3,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Imad Charkani El-Hassani,1,Très mauvaise expérience,2018-08-08 18:50:25.525162,0,https://www.google.com/maps/contrib/1029760643...,Nothing,1970-01-01,NaN,NaN,NaN
4,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Abdelkarim Ajarray,3,Expérience neutre,2017-08-08 18:50:25.525163,0,https://www.google.com/maps/contrib/1158693667...,Nothing,1970-01-01,NaN,NaN,NaN


In [ ]:

# --------------------------------------------------------
# 0.5 Initialization of database
# --------------------------------------------------------
# Utilisation de la DatabaseInitializer
ADMIN_USER = 'postgres'
ADMIN_PASSWORD = 'postgres'
HOST = 'postgresql'
PORT = '5432'
DB_USER = 'conite'
DB_PASSWORD = 'conite_password'
DB_NAME = 'bank_reviews'

# --------------------------------------------------------------------------
# Database Initialization
# --------------------------------------------------------------------------
db_initializer = DatabaseInitializer(ADMIN_USER, ADMIN_PASSWORD, HOST, PORT)

db_initializer.create_database_and_user(DB_USER, DB_PASSWORD, DB_NAME)

# ------------------------------------------------------------------------------
# Configuration de la base de donnees
# ------------------------------------------------------------------------------
DB_URI = f'postgresql://{DB_USER}:{DB_PASSWORD}@{HOST}:5432/{DB_NAME}'

# ------------------------------------------------------------------------------
# Initialize database and create tables
# ------------------------------------------------------------------------------

engine = create_engine(DB_URI)
Session = sessionmaker(bind=engine)
session = Session()
wait_for_postgresql(host=HOST, port=PORT, user=DB_USER, password=DB_PASSWORD, dbname=DB_NAME)
Base.metadata.create_all(engine)

conn_params = {
    'dbname': DB_NAME,
    'user': DB_USER,
    'password': DB_PASSWORD,
    'host': HOST,
    'port': PORT
}
with psycopg2.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        """
        cur.execute(query)
        existing_tables = cur.fetchall()
        existing_tables = [table[0] for table in existing_tables]
        print("\033[1;92mTABLES CREATED SUCCESSFULLY.\033[0m \033[1;94m" + str(existing_tables) + "\033[0m")

# --------------------------------------------------------
# 0.6 Insertion of data in database
# --------------------------------------------------------
insert_data_from_dataframe(df, session)
migration_to_decisionalDB(DB_NAME, DB_PASSWORD, session)
# session.commit()